# Working with multidimensional spatial data in Python

## Overview 

In this notebook, we will use be introduced to the foundational packages for working with spatial data in python, and learn how to use these to manipulate spatial data.

## Learning Objectives
1. Use `numpy` to manipulate array data
2. Use `pandas` to manipulate tabular data
3. Apply basic geospatial manipulations to vector data using `geopandas`
3. Work with multidimensional gridded data in `xarray`
3. Perform geospatial operations on `xarray`s using `rioxarray`

## Requirements
This tutorial requires the following Python modules installed `numpy`, `pandas`,`geopandas`, `xarray`,`rioxarray`


### Foundational Python Packages

#### NumPy

NumPy is a popular library for storing arrays of numbers and performing computations on them. Not only does it enable writing more succinct code, it also makes the code faster, since most NumPy routines are implemented in C for speed.

Numpy is probably the most important package for doing data science and scientific analysis in Python. It is very powerful and is often the basis for other packages that add additional complexity (it is at the core of all raster handling packages in python)

To use NumPy in your program, you need to import it as follows

In [ ]:
import numpy as np

##### Array creation



NumPy arrays can be created from Python lists

In [ ]:
my_array = np.array([1, 2, 3])
my_array

NumPy supports arrays of arbitrary dimension. For example, we can create two-dimensional arrays (e.g. to store a matrix) as follows

In [ ]:
my_2d_array = np.array([[1, 2, 3], [4, 5, 6]])
my_2d_array

We can access individual elements of a 2d-array using two indices as indexing  works just like for Python lists

In [ ]:
# index using:
# position_in_dim1, position_in_dim2
my_2d_array[1, 2]

We can also access rows

In [ ]:
my_2d_array[1]

and columns

In [ ]:
my_2d_array[:, 2]

a bare index of `:` means give me everything in that dim

In [ ]:
print(my_2d_array[:, :])
#is the same as
print(my_2d_array)

Arrays have a `shape` attribute

In [ ]:
print(my_array.shape)
print(my_2d_array.shape)

##### Basic operations

In NumPy, we express computations directly over arrays. This makes the code much more succint.

Arithmetic operations can be performed directly over arrays. For instance, assuming two arrays have a compatible shape, we can add them as follows

In [ ]:
array_a = np.array([1, 2, 3])
array_b = np.array([4, 5, 6])
array_a + array_b

NumPy has a range of functions that can be applied to arrays, e.g. `np.sin`

In [ ]:
np.sin(array_a)

numpy arrays also have their own methods e.g matrix transpose `.transpose()`

In [ ]:
array_a.transpose()

Because numpy arrays can have many dimensions, indexing can get quite complicated. If you want to know more have a look at the docs [here](https://numpy.org/doc/stable/user/basics.indexing.html). Lets look at what this looks like on a 3D array

In [ ]:
# Create a 3D array with values ranging from 0 to 269 with shape 3x3x30.
array_3d = np.arange(270).reshape(3, 3, 30)
array_3d

Lets pretend that this array has dimension latitude, longitude and band (3x3x30). Now lets extract a spatial subset  - we only want the first 2x2 lat long grid with all wavelengths

In [ ]:
#dimensions separated by comma
#1st dim - :2 indicates we want everything up to the 2nd coordinate i.e 0 and 1. Writing 0:2 would be equivalent
#2nd dim - sanme as above
#3rd dim -  : a bare semicolon indicate we want all entires in that dimension
sliced_array = array_3d[:2, :2, :]
sliced_array

#### Pandas


Pandas is the Python library used for tabular data manipulation and analysis. It is **very** widely used. It is the Python version of dplyr, tidyr and readr rolled into one.

Here's an example of loading a DataFrame from a CSV file:

In [ ]:
import pandas as pd

# Load data from a CSV file into a DataFrame
df = pd.read_csv('https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/0e7a9b0a5d22642a06d3d5b9bcbad9890c8ee534/iris.csv')

# Display the first 5 rows of the DataFrame
print(df.head())

There are lots and lots of tutorial on how to wrnagle data with pandas. I will just show you some basics

##### Indexing
Or how to select particular rows and columns

In [ ]:
# Accessing a column
print(df['sepal_length'])

# use .loc to access a row by label
print(df.loc[0])

# use .iloc to access a row by integer index
print(df.iloc[0])

##### Filtering
or selecting rows by a logical condition


In [ ]:
# Filter rows where 'sepal_length' is greater than 5
filtered = df[df['sepal_length'] > 5]

#what is happening here is that df['sepal_length'] > 5 is creating a series of True/False values based on the condition
#this is then used to retrun rows where the series is True

print(filtered)

another way of achieving the same thing is using `query`

In [ ]:
filtered = df.query('sepal_length > 5')

print(filtered)

##### Grouping and aggregation

In [ ]:
# Group data by the values in 'species' and compute the mean of the other columns for each group
grouped = df.groupby('species').mean()
print(grouped)

In [ ]:
# or maybe we just want the aggregation of a single colmn
petal_grouped = df.groupby('species')['petal_length'].mean()
print(petal_grouped)

is the same as

In [ ]:
#we can use the .mean() method without explicitly assinging the result of the preceding function
#the trick is that you have to know the the type/class of the first function is to know what methods it has
#but with pandas the results are usually another dataframe
result = df.groupby('species').mean()

Lets see how we can chain many things together

In [ ]:
#lets do everything above in one step
result = (pd.read_csv('https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/0e7a9b0a5d22642a06d3d5b9bcbad9890c8ee534/iris.csv')
          .query('sepal_length > 5')
          .groupby('species')
          .mean())

print(result)

### Geospatial data handling
A number of packages exist for working with geospatial data in Python. Some are more widely used than others. GeoPandas is the standard for working with spatial vector data. For working with raster data traditionally Rasterio has been the standard. Rasterio is a wrapper for GDAL which you may already be familiar with. Rasterio is not well suited to working with data with more than 2 dimensions (lat/long) or with a large number of bands. Xarray is well suited to high-dimensional data, and is rapidly growing in popularity. Hence this is what we will focus on. We will also spend a fair bit of time learning about dask, a package for making xarray (and Python in general) scale to large datasets.

#### Vector data with GeoPandas
[GeoPandas](https://geopandas.org/en/stable/getting_started.html) extends the datatypes used by pandas to allow spatial operations on geometric types. If you have worked with sf in R, you will find Geopandas very familiar. Underneath, Geopandas uses GEOS for geometric calculcation via the shapely package. In R, sf also uses GEOS.

The core data structure in GeoPandas is the `geopandas.GeoDataFrame`, a subclass of `pandas.DataFrame` that can store geometry columns and perform spatial operations.
A GeoDataFrame is a combination of regular pandas columns (`pandas.Series`), with traditional data and a special geometry column (`geopandas.GeoSeries`), with geometries (points, polygons, etc.). The geometry column has a `GeoSeries.crs` attribute, which stores information about the projection

![](geopandas.png)

In this example, we will explore 2 datasets, `swfynbos.gpkg`, a dataset of the vegetation types of the southwestern Cape of South Africa, and `fynbos_remnants`, a dataset of the remaining fragments of natural vegetation in this region. This data is in the geopackage format but Geopandas can open all commonly encountered geospatial vector data formats.

In [ ]:
#we typically use the alias gpd
import geopandas as gpd

#read file
vegtypes = gpd.read_file('shared/users/gmoncrieff/data/swfynbos.gpkg')

#view some rows
vegtypes.head()

Before getting into some data manipulations, lets looks at some attributes of the data. Geopandas allows us to easlity access some relevant attributes of our data

In [ ]:
#the type of each geometry
print(vegtypes.type)

In [ ]:
#area of each polygon
print(vegtypes.area)

In [ ]:
#centroid of each polygon
print(vegtypes.centroid)

We can print the coordinate reference system of the geodataframe using `.crs`

In [ ]:
vegtypes.crs

Doing things like filters, selecting columns and rows, etc works exactly like a Pandas dataframe, as a geodataframe is a subclass of a dataframe

In [ ]:
#select first 5 rows
print(vegtypes.iloc[0:5])

In [ ]:
#filter to a single vegtypes
print(vegtypes.query('Name_18 == "Hangklip Sand Fynbos"').head())


Plotting is easy too. Like Pandas there is a handy `.plot()` method for geodataframes.

In [ ]:
#colour plot by vegetation type
vegtypes.plot('Name_18')

Before we start playing with maniplating geodataframes based on their geometries, let's load another dataset that we will combine with the first

In [ ]:
remnants = gpd.read_file('data/remnants.gpkg')
#lets view the first few rows
remnants.head()

Set operations like intersections and unions can be applied using the `gpd.overlay()` function. Let's extract the remaining natural vegetation of each vegetation type

In [ ]:
#intersection of vegtypes and remnants
veg_remnants = gpd.overlay(vegtypes,remnants,how='intersection')

#plot!
veg_remnants.plot('Name_18')

When executing set operations, the properties from both input dataframes are retained, so each row in the output will have all the columns from the inputs

In [ ]:
veg_remnants.head()

finally, lets combine all polygons with the same threat status together using `dissolve` to simplify our geodataframe

In [ ]:
#all polygons with the saem threat status into one
veg_remnants_simple = veg_remnants.dissolve('RLE2021')
#view
veg_remnants_simple.head()

There is tons more functionality in GeoPandas, you can spatially join geodatframes with `.sjoin()`, reproject using `to_crs()`, and do all the good stuff you would expect. Two great places to dive deeper are the [GeoPandas user guide](https://geopandas.org/en/stable/docs/user_guide.html), and [the Carpentries lesson on vector data in Python](https://carpentries-incubator.github.io/geospatial-python/07-vector-data-in-python/index.html)

#### Gridded data with Xarray

##### xarray concepts

[Xarray](https://docs.xarray.dev/en/stable/) is the meat and potatoes of working with multidimensional gridded data in Python. While numpy provides many of the core operations we need for working with gridded data like indexing  matrix operations, etc it does not provide the functionality to add information about the various dimensions of arrays, the coordinates of grid cells, or attached important metadata. This is where Xarray comes in.

By including labels on array dimensions Xarray opens up many new possibilities:

- applying operations over dimensions by name: x.sum('time').

- selecting values by label x.sel(time='2014-01-01').

- use the split-apply-combine paradigm with groupby: x.groupby('time.dayofyear').mean().

- keeping track of arbitrary metadata in the form of a Python dictionary: x.attrs.

- and much more

The Xarray data structure makes it trivial to go from 2 to 3 to 4 to N dimensions, hence it is a great choice for working with imaging spectroscopy data where we will have at least 3 (lat, lon, wavelength) dimensions. Another big benefit is that it seamlessly integrates with `Dask` a popular library for parallel computing in Python. This allows us to scale analysis with Xarray to very large data.



The core data structure of Xarray is an `xarray.DataArray` - which in its simplest form is just a Numpy array with named dimensions and coordinates on those dimensions. We can combine multiple `xarray.DataArray` in a single structure called a `xarray.Dataset`. Let's see what this looks like

In [ ]:
#typically we use the xr aliais
import xarray as xr
import numpy as np

#create a 2x3 np array
arr = np.random.randn(2, 3)

#create a xarray.DataArray by naming the dims and giving them coordinates
xda = xr.DataArray(arr,
                    dims=("x", "y"),
                    coords={"x": [10, 20],
                            "y": [1.1,1.2,1.3]})

xda

We can access the individual components like the data itself, the dimension names or the coordinates using accessors

In [ ]:
#get the actual data
print(xda.values)

In [ ]:
#get teh dimenson names
print(xda.dims)

In [ ]:
#get the x coordinates
print(xda.x)

We can set or get any metadata attribute we like

In [ ]:
xda.attrs["long_name"] = "random mesurement"
xda.attrs["random_attribute"] = 123

print(xda.attrs)

and perform calculations on `xarray.DataArrays` as if they were Numpy arrays

In [ ]:
xda + 10

In [ ]:
np.sin(xda)

An `xarray.Dataset` is a container of multiple aligned DataArray objects

In [ ]:
#create a new dataarray with aligned dimensions (but it can be more or fewer dims)
#create a new 2x3x4 xarray Dataarray
arr2 = np.random.randn(2, 3, 4)
xda2 = xr.DataArray(arr2,
                    dims=("x", "y","z"),
                    coords={"x": [10, 20],
                            "y": [1.1,1.2,1.3],
                            "z": [20,200,2000,20000]})

#combine with another xarray.DataArray to make a xarray.Dataset
xds = xr.Dataset({'foo':xda,'bar':xda2})
xds


Here you can see that we have multiple arrays in a single dataset. Xarray automatically aligns the arrays based on shared dimensions and coodrinates. You can do almost everything you can do with DataArray objects with Dataset objects (including indexing and arithmetic) if you prefer to work with multiple variables at once. You can also easily retrieve a single DataArray by name from a Dataset

In [ ]:
xds.foo
# xds['foo'] works the same

##### Terminology
It is important to be precise with our terminology when dealing with Xarrays as things can quickly get confusing when working with many dims. The full glossary can be found [here](https://docs.xarray.dev/en/stable/user-guide/terminology.html), but a quick recap:
- `xarray.DataArray` - A multi-dimensional array with labeled or named dimensions
- `xarray.Dataset` - A collection of DataArrays with aligned dimensions
- **Dimension** - The (named) axes of an array
- **Coordinate** - An array that labels a dimension

![](xarray.png)

##### loading data from files

Xarray supports reading and writing of several file formats, from simple Pickle files to the more flexible netCDF format. The recommended way to store Xarray data structures is netCDF. Xarray is based on the netCDF data model, so netCDF files on disk directly correspond to Dataset objects. If you aren’t familiar with this data format, the [netCDF FAQ](https://www.unidata.ucar.edu/software/netcdf/docs/faq.html#What-Is-netCDF) is a good place to start. When we are working with complex multidimensional data, file formats start to matter a lot, and they make a big difference to how fast and efficiently we can load and analyse data. More on this in the next lesson.

We can load netCDF files to create a new Dataset using `open_dataset()`. Similarly, a DataArray can be saved to disk using the `DataArray.to_netcdf()` method

For the rest of this lesson we will work with a small dataset from a [Specim FENIX](https://www.specim.com/products/fenix/) airbone imaging spectrometer collected near the town of Franschoek, near Cape Town in 2018. Lots of important metadata about the image has been removed to keep this simple. All that remains are the measured reflectances, and the latitude, longitude and wavelength coordinates.

In [ ]:
xda_is = xr.open_dataset("shared/users/gmoncrieff/data/is_example.nc")
xda_is

##### indexing, selecting and masking

While you can use numpy-like indexing e.g `da[:,:]`, this does not make use of the power of having named dims and coords. Xarray as specific method for selecting using the position in the array `.isel()` and using the coordinates with `.sel()`

In [ ]:
#idexing using position
xda_is.isel(x=20,y=20)

We can extract a continous slice of an array dimension using `slice()`

In [ ]:
xda_is.isel(x=20,y=20,wl=slice(0,20))

We can use all the same techniques, but provive coordinate values rather than positions if we use `.sel()`. We can also provide an option for what to do if we do not get an exact match to the provided coordinates.

In [ ]:
xda_is.sel(x=3.175e+05,y=6.263e+06,method='nearest')

We can mask values in our array using conditions based on the array values or coordinate values with `.where()`

In [ ]:
# drop bad bands
xda_is = xda_is.where(xda_is.wl < 2.1,drop=True)
xda_is

You may notice that often it takes almost no time at all to run xarray code. This is because for many functions xarray does not load data from disk and actually perform the calculation, rather it simply prints a summary and high-level overview of the data that will be produced. This is called **Lazy computation** and is the smart thing to do when working with large datasets. Only when you really need to do the calculation does it actually happen - like when calling `.plot()` or writing results. We can force computation by running `xarray.DataArray.compute()`

In [ ]:
xda_is.compute()

##### Chunks
When opening our data we can specific that we want the data split into chunks along each dimension like this:

In [ ]:
xda_chunk = xr.open_dataset("data/is_example.nc",chunks={'x':50,'y':50,'wl':-1})
xda_chunk

##### What does this do, and why should we do it?
If you don't specify that you want the dataset chunked, xarray will load all the data into a numpy array. This can be okay if you are working witha small dataset but as your data grows larger chunking has a number of advantages:
 
 - __Efficient Memory Usage__
 Without chunking, xarray loads the entire dataset into memory as NumPy arrays, which can use a lot of RAM and may cause your system to slow down or crash. Chunking splits the data into smaller pieces, allowing you to work with datasets that are bigger than your available memory by loading only what you need.
 
 - __Better Performance__
Processing smaller chunks can speed up computations and make data handling more efficient.Data is loaded into memory only when required, reducing unnecessary memory usage and improving processing speed.

Checkout the [dask documentation on chunks](https://docs.dask.org/en/latest/array-chunks.html) to find out more about chunking your array.

##### Default chunking and rechunking
Some file types like netCDF or zarr have native chunking, and it is usually most efficient to use the chunking that is already present. If you specify `chunks='auto'` chunking will be automatically determined. This is a major advantage as chunking/rechunking can be expensive for large files. The downside is that you are subject to the chunking chosen by the creator of the file. 

You will remember that forcing computation using `compute()` returned a Numpy array. If we want to force computation but keep the resulting array as a chunked array we can use `persist()` instead.

In [ ]:
#the example from above
xda_chunk = xda_chunk.where(xda_chunk.wl < 2.1,drop=True)
#persist instead of compute
xda_chunk.persist()


#### Make xarray geospatial with rioxarray

Although we have latitude and longitude values associated with our Xarray, this data is not a proper geospatial dataset and hence we cannot do spatial manipulations like calculating distances or reprojecting. Xarray is a general-purpose tool for any multidimensional data and is not specific to geospatial data. We need an additional package `rioxarray` which brings all of the power of `GDAL` to Xarrays. `rioxarray` extends Xarray with the `rio` accessor. What this means is that a bunch of new functions become available to Xarray instances by typing `xarray.DataArray.rio.` 

In [ ]:
import rioxarray

The first and most important detail we need to add before turning our Xarray into a geospatial dataset is information about the projection. Here we know the current crs is epsg:32734 (UTM zone 34S) 

In [ ]:
xda_chunk = xda_chunk.rio.write_crs('epsg:32734')
xda_chunk.rio.crs

In [ ]:
xda_chunk

Now that we know the current projection, it is easy to reproject to match the projection of the vector data we were working with earlier

In [ ]:
xda_chunk_wgs = xda_chunk.rio.reproject('epsg:4326')
xda_chunk_wgs

rioxarray gives us the ability to perform a numbr of spatial operations e.g. clip and mosaict, and read and write any file format supported by GDAL. This is as simple as `rioxarray.open_rasterio()` and `xarray.DataArray.rio.to_raster()`

### credits:

This lesson has borrowed heavily from:  

[https://github.com/data-psl/lectures2020](https://github.com/data-psl/lectures2020)

[The Carpentries Geospatial Python lesson by Ryan Avery](https://carpentries-incubator.github.io/geospatial-python/)  

[The geopandas user guide](https://geopandas.org/en/stable/docs/user_guide.html)  

[The xarray user guide](https://docs.xarray.dev/en/stable/user-guide/index.html) 

[An Introduction to Earth and Environmental Data Science](https://earth-env-data-science.github.io/intro.html)
